# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *"A non-local algorithm for image denoising"* (autorzy: Antoni Buades, Bartomeu Coll i Jean Michel Morel) na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{2}
\end{equation}

gdzie:
\begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{3}
\end{equation}

- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując: jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

mat = loadmat('MR_data.mat')
Input = mat['I_noisy2']

def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g


def non_local_means_convulsion(Input, V_size, N_size, alpha, sigma):
    IConv = Input.copy()
    v = fgaussian(N_size, sigma)
    v = v / np.sum(v)
    X, Y = Input.shape
    for i in range(int(V_size / 2) + int(N_size / 2), X - int(V_size / 2) - int(N_size / 2)):
        for j in range(int(V_size / 2) + int(N_size / 2), Y - int(V_size / 2) - int(N_size / 2)):
            V = Input[i - int(V_size / 2): i + int(V_size / 2) + 1, j - int(V_size / 2): j + int(V_size / 2) + 1]
            W = np.zeros(V.shape)
            V_help = Input[i - int(V_size / 2) - int(N_size / 2): i + int(V_size / 2) + int(N_size / 2) + 1,
                     j - int(V_size / 2) - int(N_size / 2): j + int(V_size / 2) + 1 + int(N_size / 2)]
            N_x = Input[i - int(N_size / 2): i + int(N_size / 2) + 1, j - int(N_size / 2): j + int(N_size / 2) + 1]
            N_x_times_v = np.multiply(N_x, v)
            Z = 0
            for i_inside in range(int(N_size/2), V_size - int(N_size/2)):
                for j_inside in range(int(N_size/2), V_size - int(N_size/2)):
                    N_p = V_help[i_inside - int(N_size/2): i_inside + int(N_size/2) + 1, j_inside - int(N_size/2): j_inside + int(N_size/2) + 1]
                    N_p_times_v = np.multiply(N_p, v)
                    difference = N_p_times_v - N_x_times_v
                    w = np.sqrt(np.sum(np.power(difference, 2)))
                    w = np.exp(- np.power(w, 2) / (alpha * sigma ** 2))
                    W[i_inside - int(N_size/2) , j_inside - int(N_size/2)] = w
                    Z += w
            W = W / Z
            IConv[i,j] = np.sum(np.multiply(W, V))

    return IConv

In [ ]:
def new_pixel_color_1(surr, fil):
    return np.sum(np.multiply(fil,surr))/np.sum(fil) # Korzystam z faktu, że splot sygnałów w postaci macierzy to suma iloczynu wartości macierzy wejściowych

def classical_convulsion(Input, size, sigma_s):
    IConv = Input.copy()
    X, Y = Input.shape
    fil = fgaussian(size, sigma_s)

    for i in range(int(size/2),X-int(size/2)):
        for j in range(int(size/2),Y-int(size/2)):
            surrounding = Input[i-int(size/2):i+int(size/2)+1,j-int(size/2):j+int(size/2)+1]
            IConv[i][j] = new_pixel_color_1(surrounding, fil)

    return IConv

def gamma(y, sigma_r):
    return np.exp(- np.multiply(y,y) / (2 * np.multiply(sigma_r,sigma_r)))

def new_pixel_color_2(surr, fil, sigma_r):
    gam = gamma(np.abs(surr - np.take(surr, surr.size // 2)), sigma_r)
    new_fil = np.multiply(fil,gam)
    return np.sum(np.multiply(new_fil,surr)/np.sum(new_fil))

def bilateral_convulsion(Input, size, sigma_s, sigma_r):
    IConv = Input.copy()
    X, Y = Input.shape
    fil = fgaussian(size, sigma_s)

    for i in range(int(size/2),X-int(size/2)):
        for j in range(int(size/2),Y-int(size/2)):
            surrounding = Input[i-int(size/2):i+int(size/2)+1,j-int(size/2):j+int(size/2)+1]
            IConv[i][j] = new_pixel_color_2(surrounding, fil, sigma_r)

    return IConv

In [ ]:
from timeit import default_timer as timer

start = timer()
classical = classical_convulsion(Input, 11, 2)
end = timer()
print('Czas wykonywania algorytmu klasycznej konwolucji: ', end - start)
start = timer()
bilateral = bilateral_convulsion(Input, 11, 2, 20)
end = timer()
print('Czas wykonywania algorytmu konwolucji bilateralnej: ', end - start)
start = timer()
non_local_means = non_local_means_convulsion(Input, 11, 5, 10, 2)
end = timer()
print('Czas wykonywania algorytmu konwolucji Non-Local Means: ', end - start)

fig, ax = plt.subplots(2,2, figsize=(20,20))
ax[0][0].imshow(Input, 'gray')
ax[0][0].axis('off')
ax[0][0].set_title('Obraz oryginalny')
ax[0][1].imshow(classical, 'gray')
ax[0][1].axis('off')
ax[0][1].set_title('Obraz po klasycznej konwolucji')
ax[1][0].imshow(bilateral, 'gray')
ax[1][0].axis('off')
ax[1][0].set_title('Obraz po konwolucji bilateralnej')
ax[1][1].imshow(non_local_means, 'gray')
ax[1][1].axis('off')
ax[1][1].set_title('Obraz po konwolucji Non-Local Means')
plt.show()

Jak widać, najgorsze efekty daje filtracja z użyciem filtra Gaussa, która w tym przypadku zbyt mocno rozmywa obraz, sprawiając, że krawędzie i szczegóły są słabo widoczne. Filtracja bilateralna spisuje się dobrze jedynie w obrębie jednego najmniej zaszumionego obszaru obrazu. Najlepsze efekty daje filtracja z użyciem metody Non-Local-Means, powodując usunięcie szumów praktycznie na całym obrazie. Dodatkowo krawędzie i szczegóły stają się bardziej widoczne.

Jakość zdjęcia po filtracji jest niestety w tym przypadku proporcjonalna do czasu działania algorytmu filtrującego. Konwolucja biliteralna zajmuje ponad dwa razu dłużej od filtracji Gaussa, a filtracja przy użyciu algorytmu Non-Local Means jest aż ponad 25 razy dłuższa od filtracji biliteralnej.